## Multiprocessing marching cube

reference: https://github.com/fogleman/sdf/blob/main/sdf/mesh.py#L26

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import meshplot as mp

In [ ]:
def sphere_sdf(p, center=0, radius=1):
    return np.linalg.norm(p - center, axis=1) - radius

In [ ]:
from functools import partial
from multiprocessing.pool import ThreadPool
import multiprocessing
import itertools
import time

WORKERS = multiprocessing.cpu_count()

def _cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

def _worker(sdf, job):
    X, Y, Z = job
    # P = np.vstack(np.meshgrid(X,Y,Z)).reshape(3,-1).T
    P = _cartesian_product(X, Y, Z)
    volume = sdf(P).reshape((len(X), len(Y), len(Z)))
    verts, faces, normals, values = measure.marching_cubes(volume, 0)
    # verts = verts[faces]
    scale = np.array([X[1] - X[0], Y[1] - Y[0], Z[1] - Z[0]])
    offset = np.array([X[0], Y[0], Z[0]])
    # verts = verts * scale + offset
    # verts = verts[faces]
    return verts*scale+offset, faces

X = np.linspace(-1, 1, 401)
Y = np.linspace(-1, 1, 401)
Z = np.linspace(-1, 1, 401)

batch_size = 101
s = batch_size
Xs = [X[i:i+s+1] for i in range(0, len(X), s)]
Ys = [Y[i:i+s+1] for i in range(0, len(Y), s)]
Zs = [Z[i:i+s+1] for i in range(0, len(Z), s)]

batches = list(itertools.product(Xs, Ys, Zs))
num_batches = len(batches)
num_samples = sum(len(xs) * len(ys) * len(zs) for xs, ys, zs in batches)
print(num_batches, num_samples)

pool = ThreadPool(WORKERS)
f = partial(_worker, sphere_sdf) # _worker, sdf callables

verts_combined = []
faces_combined = []

for verts, faces in pool.imap(f, batches):
    faces_combined.extend(np.array(faces) + len(verts_combined))
    verts_combined.extend(np.array(verts))
    
    
mp.plot(np.array(verts_combined), np.array(faces_combined))

64 65939264


ValueError: Surface level must be within volume data range.

In [ ]:
from functools import partial
from multiprocessing.pool import ThreadPool
import multiprocessing
import itertools
import time
import torch.nn as nn
import torch
WORKERS = multiprocessing.cpu_count()


# autodecoder MLP class
class MLP(nn.Module):
    def __init__(self, n_shapes, shape_code_length, n_inner_nodes):
        super(MLP, self).__init__()
        self.shape_code_length = shape_code_length
        self.shape_codes = nn.Embedding(n_shapes, shape_code_length, max_norm=0.01) # shape code as an embedding # TODO: take this outside 
        self.linear1 = nn.Linear(3 + shape_code_length, n_inner_nodes) # (x, y, z) + shape code 
        self.linear2 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear3 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear4 = nn.Linear(n_inner_nodes, 1) # output a SDF value
        self.relu = nn.ReLU()

    def forward(self, shape_idx, x):
        shape_code = self.shape_codes(shape_idx.view(1, -1))
        shape_code = shape_code.view(-1, self.shape_code_length)
        shape_code_with_xyz = torch.cat((x, shape_code), dim=1) # concatenate horizontally
        
        out = self.linear1(shape_code_with_xyz)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.linear4(out)
        
        return out
    
model = MLP(10, 256, 256)
model.load_state_dict(torch.load('./models/autodecoder_08052022_073446'))
model.eval()


shape_idx = 0


def _cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

def _worker(sdf, job):
    X, Y, Z = job
    # P = np.vstack(np.meshgrid(X,Y,Z)).reshape(3,-1).T
    P = _cartesian_product(X, Y, Z)
    P = torch.from_numpy(P)
    shape_idx_tensor = torch.ones((P.shape[0], 1), dtype=torch.int) * shape_idx
    volume = sdf(shape_idx_tensor, P).reshape((len(X), len(Y), len(Z)))
    verts, faces, normals, values = measure.marching_cubes(volume, 0)
    volume = volume.detach().numpy()
    # verts = verts[faces]
    scale = np.array([X[1] - X[0], Y[1] - Y[0], Z[1] - Z[0]])
    offset = np.array([X[0], Y[0], Z[0]])
    # verts = verts * scale + offset
    # verts = verts[faces]
    return verts*scale+offset, faces

X = np.linspace(-1, 1, 201)
Y = np.linspace(-1, 1, 201)
Z = np.linspace(-1, 1, 201)

batch_size = 11
s = batch_size
Xs = [X[i:i+s+1] for i in range(0, len(X), s)]
Ys = [Y[i:i+s+1] for i in range(0, len(Y), s)]
Zs = [Z[i:i+s+1] for i in range(0, len(Z), s)]

batches = list(itertools.product(Xs, Ys, Zs))
num_batches = len(batches)
num_samples = sum(len(xs) * len(ys) * len(zs) for xs, ys, zs in batches)
print(num_batches, num_samples)

pool = ThreadPool(WORKERS)
f = partial(_worker, model) # _worker, sdf callables

verts_combined = []
faces_combined = []

for verts, faces in pool.imap(f, batches):
    faces_combined.extend(np.array(faces) + len(verts_combined))
    verts_combined.extend(np.array(verts))
    
    
mp.plot(np.array(verts_combined), np.array(faces_combined))

FileNotFoundError: [Errno 2] No such file or directory: './models/audodecoder_08052022_073446'